## Iniciando Modelo de Entrenamiento
Para esta parte, el articulo menciona el uso de modelo orientado a Random Forest con Regresion, actualmente ya se tiene el dataset final con mas de 34000 registros y 72 caracteristicas.
Se utilizara TensorFlow corriendo con Cuda para el entrenamiento y testeo

In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

2024-11-24 22:58:52.461077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732507132.477538    6928 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732507132.482446    6928 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 22:58:52.501307: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
print(tf.__version__)

2.18.0


In [3]:
import torch

print(torch.cuda.is_available())

True


### Importamos el dataset

In [4]:
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np

In [5]:
dataset_df = pd.read_csv("main_dataset.csv")
dataset_df.head()

,datetime,glucose,glucose_24h_mean,nivel,gender,hba1c,id,acc_mean,acc_median,acc_max,...,food2h_sugar_sum,food2h_sugar_mean,food2h_protein_sum,food2h_protein_mean,food2h_cantidad_comidas_,food2h_media_comidas_,food8h_cantidad_comidas_,food8h_media_comidas_,food24h_cantidad_comidas_,food24h_media_comidas_
0,2020-02-13 17:20:00,61.0,101.45,PersNorm,FEMALE,5.5,1.0,64.328161,63.976558,85.545310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-02-13 17:25:00,59.0,101.45,PersNorm,FEMALE,5.5,1.0,64.632465,63.921827,90.365923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-02-13 17:30:00,58.0,101.45,PersNorm,FEMALE,5.5,1.0,64.531001,64.148266,88.345911,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-02-13 17:35:00,59.0,101.45,PersNorm,FEMALE,5.5,1.0,63.708273,63.095166,149.298359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-13 17:40:00,63.0,101.45,PersNorm,FEMALE,5.5,1.0,63.475463,63.063460,157.711762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
classes = dataset_df['nivel'].unique().tolist()
print(classes)

['PersNorm', 'PersHigh', 'PersLow']


In [7]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [8]:
train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

24136 examples in training, 10456 examples for testing.
